

# Tune Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

# Gridsearch Hyperparameters

In the guided project, you learned how to use sklearn's GridsearchCV and keras-tuner library to tune the hyperparamters of a neural network model. For your module project you'll continue using these two libraries however we are going to make things a little more interesting for you. 

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. 



**Don't forgot to switch to GPU on Colab!**

In [51]:
# native python libraries imports 
import math
from time import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# sklearn imports 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# keras imports 
import keras
from keras import Sequential
from keras.layers import Dense
from kerastuner.tuners import RandomSearch, BayesianOptimization, Sklearn
from kerastuner.engine.hyperparameters import HyperParameters
from keras.activations import relu, sigmoid
from keras.optimizers import Adam, SGD
from keras.utils import get_file

# required for compatibility between sklearn and keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
def load_quickdraw10():
    """
    Fill out this doc string, and comment the code, for practice in writing the kind of code that will get you hired. 
    """
    
    URL_ = "https://github.com/LambdaSchool/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/quickdraw10.npz?raw=true"
    
    path_to_zip = get_file('./quickdraw10.npz', origin=URL_, extract=False)

    data = np.load(path_to_zip)
    
    # normalize your image data
    max_pixel_value = 255
    X = data['arr_0']/max_pixel_value
    Y = data['arr_1']
        
    return train_test_split(X, Y, shuffle=True)

In [3]:
X_train, X_test, y_train, y_test = load_quickdraw10()

25427968/25421363 [==============================] - 0s 0us/step


In [4]:
X_train.shape

(75000, 784)

In [5]:
y_train.shape

(75000,)

_____

# Experiment 1

## Tune Hyperperameters using Enhanced GridsearchCV 

We are going to use GridsearchCV again to tune a deep learning model however we are going to add some additional functionality to our gridsearch. Specifically, we are going to automate away the generation of how many nodes to use in a layer and how many layers to use in a model! 

By the way, yes, there is a function within a function. Try to not let that bother you. An alternative to this would be to create a class. If you're up for the challenge give it a shot. However, consider this a stretch goal that you come back to after you finish going through this assignment. 


### Objective 

The objective of this experiment is to show you how to automate the generation of layers and layer nodes for the purposes of gridsearch. Up until now, we've been manually selecting the number of layers and layer nodes.

In [6]:
# Function to create model, required for KerasClassifier
def create_model(n_layers,  first_layer_nodes, last_layer_nodes, act_funct ="relu", negative_node_incrementation=True):
    """"
    Returns a complied keras model 
    
    Parameters
    ----------
    n_layers: int 
        number of hidden layers in model 
        To be clear, this excludes the input and output layer.
        
    first_layer_nodes: int
        Number of nodes in the first hidden layer 

    last_layer_nodes: int
        Number of nodes in the last hidden layer (this is the layer just prior to the output layer)
        
     act_funct: string 
         Name of activation function to use in hidden layers (this excludes the output layler)
        
    Returns
    -------
    model: keras object 
    """
    
    def gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation=True):
        """
        Generates and returns the number of nodes in each hidden layer. 
        To be clear, this excludes the input and output layer. 

        Note
        ----
        Number of nodes in each layer is linearly incremented. 
        For example, gen_layer_nodes(5, 500, 100) will generate [500, 400, 300, 200, 100]

        Parameters
        ----------
        n_layers: int
            Number of hidden layers
            This values should be 2 or greater 

        first_layer_nodes: int

        last_layer_nodes: int

        Returns
        -------
        layers: list of ints
            Contains number of nodes for each layer 
        """

        # throws an error if n_layers is less than 2 
        assert n_layers >= 2, "n_layers must be 2 or greater"

        layers = []

        # PROTIP: IF YOU WANT THE NODE INCREMENTATION TO BE SPACED DIFFERENTLY
        # THEN YOU'LL NEED TO CHANGE THE WAY THAT IT'S CALCULATED - HAVE FUN!
        # when set to True number of nodes are decreased for subsequent layers 
        if negative_node_incrementation:
            # subtract this amount from previous layer's nodes in order to increment towards smaller numbers 
            nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
            
        # when set to False number of nodes are increased for subsequent layers
        else:
            # add this amount from previous layer's nodes in order to increment towards larger numbers 
            nodes_increment = (first_layer_nodes - last_layer_nodes)/ (n_layers-1)

        nodes = first_layer_nodes

        for i in range(1, n_layers+1):

            layers.append(math.ceil(nodes))

            # increment nodes for next layer 
            nodes = nodes + nodes_increment

        return layers
    
    # create model
    model = Sequential()
    
    n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
    
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=act_funct))
        else:
            model.add(Dense(n_nodes[i-1], activation=act_funct))
            
            
    # output layer 
    model.add(Dense(10, # 10 unit/neurons in output layer because we have 10 possible labels to predict  
                    activation='softmax')) # use softmax for a label set greater than 2            
    
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', # adam is a good default optimizer 
                  metrics=['accuracy'])
    
    # do not include model.fit() inside the create_model function
    # KerasClassifier is expecting a complied model 
    return model


## Explore create_model

Let's build a few different models in order to understand how the above code works in practice. 

### Build model 

Use `create_model` to build a model. 

- Set `n_layers = 10` 
- Set `first_layer_nodes = 500`
- Set `last_layer_nodes = 100`
- Set `act_funct = "relu"`
- Make sure that `negative_node_incrementation = True`

In [7]:
# use create_model to create a model 

# YOUR CODE HERE
model = create_model(
                     n_layers=10,
                     first_layer_nodes=500,
                     last_layer_nodes=100,
                     act_funct="relu",
                     negative_node_incrementation=True
                     )

In [8]:
# run model.summary() and make sure that you understand the model architecture that you just built 
# Notice in the model summary how the number of nodes have been linearly incremented in decreasing values. 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               392500    
_________________________________________________________________
dense_1 (Dense)              (None, 456)               228456    
_________________________________________________________________
dense_2 (Dense)              (None, 412)               188284    
_________________________________________________________________
dense_3 (Dense)              (None, 367)               151571    
_________________________________________________________________
dense_4 (Dense)              (None, 323)               118864    
_________________________________________________________________
dense_5 (Dense)              (None, 278)               90072     
_________________________________________________________________
dense_6 (Dense)              (None, 234)               6

### Build model 

Use `create_model` to build a model. 

- Set `n_layers = 10` 
- Set `first_layer_nodes = 500`
- Set `last_layer_nodes = 100`
- Set `act_funct = "relu"`
- Make sure that `negative_node_incrementation = False`

In [9]:
# use create_model to create a model 

# YOUR CODE HERE
model = create_model(
                     n_layers=10,
                     first_layer_nodes=500,
                     last_layer_nodes=100,
                     act_funct="relu",
                     negative_node_incrementation=False
                     )

In [10]:
# run model.summary() and make sure that you understand the model architecture that you just built 
# Notice in the model summary how the number of nodes have been linearly incremented in increasing values.
# The output layer must have 10 nodes because there are 10 labels to predict 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 500)               392500    
_________________________________________________________________
dense_11 (Dense)             (None, 545)               273045    
_________________________________________________________________
dense_12 (Dense)             (None, 589)               321594    
_________________________________________________________________
dense_13 (Dense)             (None, 634)               374060    
_________________________________________________________________
dense_14 (Dense)             (None, 678)               430530    
_________________________________________________________________
dense_15 (Dense)             (None, 723)               490917    
_________________________________________________________________
dense_16 (Dense)             (None, 767)              

In [11]:
# feel free to play around with parameters to gain additional insight as to how the create_model function works 
model = create_model(
                     n_layers=10,
                     first_layer_nodes=250,
                     last_layer_nodes=7000,
                     act_funct="softmax",
                     negative_node_incrementation=True
                     )

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 250)               196250    
_________________________________________________________________
dense_21 (Dense)             (None, 1000)              251000    
_________________________________________________________________
dense_22 (Dense)             (None, 1750)              1751750   
_________________________________________________________________
dense_23 (Dense)             (None, 2500)              4377500   
_________________________________________________________________
dense_24 (Dense)             (None, 3250)              8128250   
_________________________________________________________________
dense_25 (Dense)             (None, 4000)              13004000  
_________________________________________________________________
dense_26 (Dense)             (None, 4750)             

Ok, now that we've played around a bit with  `create_model` in order to understand how it works, let's build a much simpler model that we'll be running gridsearches. 

### Build model 

Use `create_model` to build a model. 

- Set `n_layers = 2` 
- Set `first_layer_nodes = 500`
- Set `last_layer_nodes = 100`
- Set `act_funct = "relu"`
- Make sure that `negative_node_incrementation = True`

In [12]:
# use create_model to create a model 

# YOUR CODE HERE
model = create_model(n_layers=2,
                     first_layer_nodes=500,
                     last_layer_nodes=100,
                     act_funct="relu",
                     negative_node_incrementation=True)

In [13]:
# run model.summary() and make sure that you understand the model architecture that you just built 
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 500)               392500    
_________________________________________________________________
dense_31 (Dense)             (None, 10)                5010      
Total params: 397,510
Trainable params: 397,510
Non-trainable params: 0
_________________________________________________________________


In [14]:
# define the grid search parameters
param_grid = {'n_layers': [2, 3],
              'epochs': [3], 
              "first_layer_nodes": [500, 300],
              "last_layer_nodes": [100, 50]
             }

In [15]:
model = KerasClassifier(create_model)

In [16]:
# Create Grid Search
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)

grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Epoch 1/3
2344/2344 [==============================] - 4s 2ms/step - loss: 0.7649 - accuracy: 0.7693
Epoch 2/3
2344/2344 [==============================] - 4s 2ms/step - loss: 0.4329 - accuracy: 0.8711
Epoch 3/3
2344/2344 [==============================] - 4s 2ms/step - loss: 0.3422 - accuracy: 0.8975
Best: 0.8685333331425985 using {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 2}
Means: 0.8685333331425985, Stdev: 0.0011172982269957467 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 2}
Means: 0.8625199993451437, Stdev: 0.0010291597246197368 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 3}
Means: 0.86735999584198, Stdev: 0.0024967280237471893 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 50, 'n_layers': 2}
Means: 0.8658399979273478, Stdev: 0.004008138164885982 with: {'epochs': 3, 'first_layer_nodes': 500, 'l

In [17]:
best_model = grid_result.best_estimator_

In [18]:
best_model.get_params()

{'epochs': 3,
 'first_layer_nodes': 500,
 'last_layer_nodes': 100,
 'n_layers': 2,
 'build_fn': <function __main__.create_model(n_layers, first_layer_nodes, last_layer_nodes, act_funct='relu', negative_node_incrementation=True)>}

-----

# Experiment 2

## Benchmark different Optimization Algorithms 

In this section, we are going to use the same model and dataset in order to benchmark 3 different gridsearch approaches: 

- Random Search
- Bayesian Optimization. 
- Brute Force Gridsearch

Our goal in this experiment is two-fold. We want to see which appraoch 

- Scores the highest accuracy
- Has the shortest run time 

We want to see how these 3 gridsearch approaches handle these trade-offs and to give you a sense of those trades offs.

### Trade-offs

`Brute Force Gridsearch` will train a model on every single unique hyperparameter combination, this guarantees that you'll get the highest possible accuracy from your parameter set but your gridsearch might have a very long run-time. 

`Random Search` will randomly sample from your parameter set which, depending on how many samples, the run-time might be significantly cut down but you might or might not sample the parameters that correspond to the heightest possible accuracies. 

`Bayesian Optimization` has a bit of intelligence built into it's search algorithm but you do need to manually select some parameters which greatly influence the model learning outcomes. 

-------
### Build our model

In [19]:
# because gridsearching can take a lot of time and we are bench marking 3 different approaches
# let's build a simple model to minimize run time 

def build_model(hp):
    
    """
    Returns a complied keras model ready for keras-tuner gridsearch algorithms 
    """
    
    model = Sequential()
    
    # hidden layer
    model.add(Dense(units=hp.get('units'),activation=hp.get("activation")))
    
    # output layer
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer=keras.optimizers.Adam(hp.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model
  

In [20]:
# build out our hyperparameter dictionary 
hp = HyperParameters()
hp.Int('units', min_value=32, max_value=512, step=32)
hp.Choice('learning_rate',values=[1e-1, 1e-2, 1e-3])
hp.Choice('activation',values=["relu", "sigmoid"])

'relu'

------
# Run the Gridsearch Algorithms 

### Random Search

Be sure to check out the [**docs for Keras-Tuner**](https://keras-team.github.io/keras-tuner/documentation/tuners/). Here you can read about the input parameters for the `RandomSearch` tuner.

In [22]:
# how many unique hyperparameter combinations do we have? 
# HINT: take the product of the number of possible values for each hyperparameter 
# save your answer to n_unique_hparam_combos

# YOUR CODE HERE
n_unique_hparam_combos = len(range(32,512+32, 32)) * 3 *2

In [23]:
# how many of these do we want to randomly sample?
# let's pick 25% of n_unique_hparam_combos param combos to sample
# save this number to n_param_combos_to_sample

# YOUR CODE HERE
n_param_combos_to_sample = n_unique_hparam_combos * .25

In [24]:
random_tuner = RandomSearch(
            build_model,
            objective='val_accuracy',
            max_trials=n_param_combos_to_sample, # number of times to sample the parameter set and build a model 
            seed=1234,
            hyperparameters=hp, # pass in our hyperparameter dictionary
            directory='./keras-tuner-trial',
            project_name='random_search')

In [25]:
# take note of Total elapsed time in print out
random_tuner.search(X_train, y_train,
                    epochs=3,
                    validation_data=(X_test, y_test))

Trial 24 Complete [00h 00m 11s]
val_accuracy: 0.695360004901886

Best val_accuracy So Far: 0.866159975528717
Total elapsed time: 00h 04m 49s
INFO:tensorflow:Oracle triggered exit


In [26]:
# identify the best score and hyperparamter (should be at the top since scores are ranked)
random_tuner.results_summary()

Results summary
Results in ./keras-tuner-trial/random_search
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 352
learning_rate: 0.001
activation: relu
Score: 0.866159975528717
Trial summary
Hyperparameters:
units: 480
learning_rate: 0.001
activation: sigmoid
Score: 0.8622000217437744
Trial summary
Hyperparameters:
units: 384
learning_rate: 0.001
activation: relu
Score: 0.8620399832725525
Trial summary
Hyperparameters:
units: 416
learning_rate: 0.001
activation: sigmoid
Score: 0.8594800233840942
Trial summary
Hyperparameters:
units: 448
learning_rate: 0.001
activation: sigmoid
Score: 0.8570399880409241
Trial summary
Hyperparameters:
units: 288
learning_rate: 0.001
activation: sigmoid
Score: 0.8557999730110168
Trial summary
Hyperparameters:
units: 224
learning_rate: 0.001
activation: sigmoid
Score: 0.8515599966049194
Trial summary
Hyperparameters:
units: 192
learning_rate: 0.01
activation: sigmoid
Score: 0.8370400071144104
Tria

 ### Results
 
Identify and write the the best performing hyperparamter combination and model score. 
Note that because this is Random Search, multiple runs might have slighly different outcomes. 
 
 

Trial summary
---

**Hyperparameters:**

units: 352

learning_rate: 0.001

activation: relu

Score: 0.866159975528717

------
### Bayesian Optimization

![](https://upload.wikimedia.org/wikipedia/commons/0/02/GpParBayesAnimationSmall.gif)

Be sure to check out the [**docs for Keras-Tuner**](https://keras-team.github.io/keras-tuner/documentation/tuners/). Here you can read about the input parameters for the `BayesianOptimization` tuner.

Pay special attention to these `BayesianOptimization` parameters: `num_initial_points` and `beta`. 

`num_initial_points`: 

Number of randomly selected hyperparameter combinations to try before applying bayesian probability to determine liklihood of which param combo to try next based on expected improvement


`beta`: 

Larger values means more willing to explore new hyperparameter combinations (analogous to searching for the global minimum in Gradient Descent), smaller values means that it is less willing to try new hyperparameter combinations (analogous to getting stuck in a local minimum in Gradient Descent). 

As a start, error on the side of larger values. What defines a small or large value you ask? That question would pull us into the mathematical intricacies of Bayesian Optimization and Gaussian Processes. For simplicity, notice that the default value is 2.6 and work from there. 

In [38]:
# we know that 24 samples is about 25% of 96 possible hyper-parameter combos
# because BO isn't random (after num_initial_points number of trails) let's see if 15 max trials gives good results
# feel free to play with any of these numbers
max_trials=15
num_initial_points=5
beta=7.0

In [39]:
bayesian_tuner = BayesianOptimization(
                    build_model,
                    objective='val_accuracy',
                    max_trials=max_trials,
                    hyperparameters=hp, # pass in our hyperparameter dictionary
                    num_initial_points=num_initial_points, 
                    beta=beta, 
                    seed=1234,
                    directory='./keras-tuner-trial',
                    project_name='bayesian_optimization_4')

INFO:tensorflow:Reloading Oracle from existing project ./keras-tuner-trial/bayesian_optimization_4/oracle.json
INFO:tensorflow:Reloading Tuner from ./keras-tuner-trial/bayesian_optimization_4/tuner0.json


In [40]:
bayesian_tuner.search(X_train, y_train,
               epochs=3,
               validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [41]:
bayesian_tuner.results_summary()

Results summary
Results in ./keras-tuner-trial/bayesian_optimization_4
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: relu
Score: 0.870639979839325
Trial summary
Hyperparameters:
units: 352
learning_rate: 0.001
activation: relu
Score: 0.8697199821472168
Trial summary
Hyperparameters:
units: 192
learning_rate: 0.001
activation: relu
Score: 0.8596400022506714
Trial summary
Hyperparameters:
units: 320
learning_rate: 0.001
activation: sigmoid
Score: 0.859279990196228
Trial summary
Hyperparameters:
units: 480
learning_rate: 0.001
activation: sigmoid
Score: 0.8572400212287903
Trial summary
Hyperparameters:
units: 160
learning_rate: 0.001
activation: sigmoid
Score: 0.8492400050163269
Trial summary
Hyperparameters:
units: 288
learning_rate: 0.01
activation: sigmoid
Score: 0.8342800140380859
Trial summary
Hyperparameters:
units: 384
learning_rate: 0.01
activation: relu
Score: 0.8201599717140198
Tr

 ### Results
 
Identify and write the the best performing hyperparamter combination and model score. 
Note that because this is  Bayesian Optimization, multiple runs might have slighly different outcomes. 
 
 

Trial summary
---
**Hyperparameters:**

units: 512

learning_rate: 0.001

activation: relu

Score: 0.870639979839325

---------
## Brute Force Gridsearch Optimization


### Populate a Sklearn compatiable parameter dictionary

In [42]:
# build out our hyperparameter dictionary
hyper_parameters = {
    # BUG Fix: cast array as list otherwise GridSearchCV will throw error
    "units": np.arange(32, 544, 32).tolist(),
    "learning_rate": [1e-1, 1e-2, 1e-3],
    "activation":["relu", "sigmoid"]
}

In [43]:
hyper_parameters

{'units': [32,
  64,
  96,
  128,
  160,
  192,
  224,
  256,
  288,
  320,
  352,
  384,
  416,
  448,
  480,
  512],
 'learning_rate': [0.1, 0.01, 0.001],
 'activation': ['relu', 'sigmoid']}

### Build a Sklearn compatiable model function

In [44]:
def build_model(units, learning_rate, activation):
    
    """
    Returns a complie keras model ready for keras-tuner gridsearch algorithms 
    """
    
    model = Sequential()
    
    # hidden layer
    model.add(Dense(units, activation=activation))
    
    # output layer
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer=Adam(learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model

In [45]:
model = KerasClassifier(build_fn = build_model)

In [46]:
# save start time 
start = time()

# Create Grid Search
grid = GridSearchCV(estimator=model, 
                    param_grid=hyper_parameters, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)

grid_result = grid.fit(X_train, y_train)

# save end time 
end = time()

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 96 candidates, totalling 288 fits
2344/2344 [==============================] - 4s 2ms/step - loss: 0.7658 - accuracy: 0.7667
Best: 0.8464266657829285 using {'activation': 'relu', 'learning_rate': 0.001, 'units': 480}
Means: 0.2100666662057241, Stdev: 0.05747394304161123 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 32}
Means: 0.2344800035158793, Stdev: 0.05555053675739537 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 64}
Means: 0.3245066702365875, Stdev: 0.013180042849100567 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 96}
Means: 0.28329333662986755, Stdev: 0.027131093078496912 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 128}
Means: 0.25201333065827686, Stdev: 0.05545650462097443 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 160}
Means: 0.2829333295424779, Stdev: 0.05391493862665466 with: {'activation': 'relu', 'learning_rate': 0.1, 'units': 192}
Means: 0.33401333292325336, Stdev: 0.015

In [47]:
# total run time 
total_run_time_in_miniutes = (end - start)/60
total_run_time_in_miniutes

3.354429002602895

In [48]:
grid_result.best_params_

{'activation': 'relu', 'learning_rate': 0.001, 'units': 480}

In [49]:
# because all other optimization approaches are reporting test set score
# let's calculate the test set score in this case 
best_model = grid_result.best_estimator_
test_acc = best_model.score(X_test, y_test)

782/782 [==============================] - 1s 1ms/step - loss: 0.5132 - accuracy: 0.8459


In [50]:
test_acc

0.8459200263023376

 ### Results
 
Identify and write the the best performing hyperparamter combination and model score. 
 
 

Best Score:
---
0.8459200263023376

**activation:** 'relu'

**learning_rate:** 0.001

**units:** 480

_______

# Conclusion

The spirit of this experiment is to expose you to the idea of benchmarking and comparing the trade-offs of various gridsearch approaches. 

Even if we did find a way to pass in the original test set into GridSearchCV, we can see that both Random Search and Bayesian Optimization are arguably better alternatives to a brute force grid search when we consider the trade-offs of run time and locating the best performing model. 

----

# Stretch Goals

- Feel free to run whatever gridserach experiments on whatever models you like!

In [52]:
import wandb
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


True

In [61]:
# this is your open playground - be free to explore as you wish
hyperparameter_defaults = hp

# Pass your defaults to wandb.init
wandb.init(config=hyperparameter_defaults)
# Access all hyperparameter values through wandb.config
config = wandb.config

# Set up your model
model = BayesianOptimization(config,
                    build_model,
                    objective = 'val_accuracy',
                    max_trials=max_trials,
                    num_initial_points=num_initial_points,
                    beta=beta,
                    seed=1234)

# Log metrics inside your training loop
for epoch in range(config["epochs"]):
    val_acc, val_loss = model.fit()
    metrics = {"validation_accuracy": val_acc,
               "validation_loss": val_loss}
    wandb.log(metrics)


TypeError: __init__() got multiple values for argument 'objective'